In [22]:
from pathlib import Path
from pyspark import SparkConf
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, explode
from pyspark.sql.types import TimestampType


In [1]:


# Save data from S3 bucket 
s3_path = f"s3a://dataminded-academy-capstone-resources/raw/open_aq/"
config = {"spark.jars.packages": "org.apache.hadoop:hadoop-aws:3.2.0", "fs.s3a.aws.credentials.provider": "com.amazonaws.auth.DefaultAWSCredentialsProviderChain"}
conf = SparkConf().setAll(config.items())
spark = SparkSession.builder.config(conf = conf).getOrCreate()
df = spark.read.json(s3_path)

Picked up JAVA_TOOL_OPTIONS:  -Xmx3489m
Picked up JAVA_TOOL_OPTIONS:  -Xmx3489m


:: loading settings :: url = jar:file:/workspace/academy-capstone/venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/gitpod/.ivy2/cache
The jars for the packages stored in: /home/gitpod/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f7da0801-d64f-4b34-b94d-0c1bf497a817;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
:: resolution report :: resolve 259ms :: artifacts dl 8ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	--------------------------------

In [2]:
df.printSchema()

root
 |-- city: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- date: struct (nullable = true)
 |    |-- local: string (nullable = true)
 |    |-- utc: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- isAnalysis: boolean (nullable = true)
 |-- isMobile: boolean (nullable = true)
 |-- location: string (nullable = true)
 |-- locationId: long (nullable = true)
 |-- parameter: string (nullable = true)
 |-- sensorType: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- value: double (nullable = true)



In [12]:
## More straight forward way from Adriana: withColumn('longitude', col('coordinates.longitude')), then drop column

## From StackOverflow
def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

    flat_df = nested_df.select(flat_cols +
                               [col(nc+'.'+c).alias(nc+'_'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    return flat_df

flat_df = flatten_df(df)
flat_df.show()

+----+-------+---------+----------+--------+--------------------+----------+---------+---------------+-----+-----+--------------------+---------------------+--------------------+--------------------+
|city|country|   entity|isAnalysis|isMobile|            location|locationId|parameter|     sensorType| unit|value|coordinates_latitude|coordinates_longitude|          date_local|            date_utc|
+----+-------+---------+----------+--------+--------------------+----------+---------+---------------+-----+-----+--------------------+---------------------+--------------------+--------------------+
|null|     BE|community|     false|   false|      Wilsele-Herent|     66110|      pm1|low-cost sensor|µg/m³|  1.7|              50.904|               4.6959|2021-02-02T22:59:...|2021-02-02T23:59:...|
|null|     BE|community|     false|   false|      Wilsele-Herent|     66110|     pm25|low-cost sensor|µg/m³|  4.1|              50.904|               4.6959|2021-02-02T22:59:...|2021-02-02T23:59:...|


In [25]:
flat_df = flat_df.withColumn("date_local" , col("date_local").cast(TimestampType()))\
       .withColumn("date_utc" , col("date_utc").cast(TimestampType()))

flat_df.show()

+----+-------+---------+----------+--------+--------------------+----------+---------+---------------+-----+-----+--------------------+---------------------+-------------------+-------------------+
|city|country|   entity|isAnalysis|isMobile|            location|locationId|parameter|     sensorType| unit|value|coordinates_latitude|coordinates_longitude|         date_local|           date_utc|
+----+-------+---------+----------+--------+--------------------+----------+---------+---------------+-----+-----+--------------------+---------------------+-------------------+-------------------+
|null|     BE|community|     false|   false|      Wilsele-Herent|     66110|      pm1|low-cost sensor|µg/m³|  1.7|              50.904|               4.6959|2021-02-02 23:59:53|2021-02-02 23:59:53|
|null|     BE|community|     false|   false|      Wilsele-Herent|     66110|     pm25|low-cost sensor|µg/m³|  4.1|              50.904|               4.6959|2021-02-02 23:59:53|2021-02-02 23:59:53|
|null|    

## Get secrets from AWS Secrets Manager
```aws secretsmanager get-secret-value --secret-id snowflake/capstone/login```

## Log-in to AWS Console
IAMAccount: datamindedacademy
User: Vanessa
Password: On Slack